# Time series heat map
* Fig. 3c

In [ ]:
figDir = '~/Hyphosphere/data/3Exp/figures'

work_dir = '~/Hyphosphere/data/3Exp/DESeq'

# phyloseq - full object without tree (includes archaea)
physeq_file = '~/Hyphosphere/data/3Exp/phyloseq/3Exp_phyloseq_thresh_woTree_experimental_woContam.rds'

In [ ]:
library(RColorBrewer)
library(vegan)
library(reshape2)
library(phyloseq)
library(gridExtra)
library(ggplot2)
library(DESeq2)
library(pheatmap)
library(tidyr)
library(dplyr)

In [ ]:
theme_pub = function (base_size = 11, base_family = "Arial") 
{
    theme_grey(base_size = base_size, base_family = base_family) %+replace% 
        theme(
            axis.line.x = element_line(size = .5),
            axis.line.y = element_line(size = .5),
            panel.background = element_rect(fill = "white", colour = NA), 
            panel.border = element_rect(fill = NA, color = "black", size = .5), 
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            strip.background = element_rect(fill = "grey85", 
                colour = "grey20"), 
            legend.key = element_rect(fill = "white", 
                colour = NA), complete = TRUE)
    }

In [ ]:
ps = readRDS(physeq_file) %>% subset_taxa(., !is.na(Phylum)) %>%
    # remove non-experimental controls
    
    subset_samples(., SampleType %in% c("CS","CH") & Experiment == 3 & !Sample == "T0R1") %>%
     filter_taxa(., function(x) max(x) > 0, TRUE)
   
sample_data(ps)$Rep = as.factor(sample_data(ps)$Rep)

sample_data(ps)$Rep = factor(sample_data(ps)$Rep)
sample_data(ps)$DAI = factor(sample_data(ps)$DAI)

ps


                                  


In [ ]:
taxa = tax_table(ps) %>% as.data.frame() %>% mutate(ASV = row.names(.))
head(taxa)

In [ ]:
sample_data(ps)$SoilxFungus = paste(sample_data(ps)$Soil,"-",sample_data(ps)$Fungus, sep = "")
sample_data(ps) %>% head()



## read back in log2foldchange files

In [ ]:
readfile = file.path(work_dir, 'dfl2fc-Exp3.txt')
df.l2fc.Exp3 = read.table(readfile, header = TRUE, sep= '\t')

readfile = file.path(work_dir, 'dfl2fc-Exp3-time.txt')
df.l2fc.Exp3.time = read.table(readfile, header = TRUE, sep= '\t')

head(df.l2fc.Exp3.time)

# Which ASVs are time specific

# Which Hyphal ASVs change with time

In [ ]:
HASVs = df.l2fc.Exp3 %>%
    filter(Contrast == "CH_CS" & padj < 0.05 & log2FoldChange > 0 & Subset == "Lansing-Gv") %>%
    .$OTU %>% 
    as.character() %>%
    unique()
length(HASVs)

df.THASVs = df.l2fc.Exp3.time %>%
    filter(SampleType == "CH" & padj < 0.05 & OTU %in% HASVs) %>%
     .$OTU %>%
    unique() %>% 
    as.character()
length(df.THASVs)

# Heat map of variance stabilized counts

### CH CS

In [ ]:
psCHCS = readRDS(physeq_file) %>% subset_taxa(., !is.na(Phylum)) %>%
    # remove non-experimental controls
    
    subset_samples(., SampleType %in% c("CS","CH") & Experiment == 3 & !Sample == "T0R1") %>%
     filter_taxa(., function(x) max(x) > 0, TRUE)
     # re-normalize after subsetting taxa
    #transform_sample_counts(., function(x) x/sum(x))
sample_data(psCHBS)$Rep = as.factor(sample_data(psCHBS)$Rep)

psCHCS


                                  


In [ ]:
sample_data(ps)$Rep = factor(sample_data(ps)$Rep)
sample_data(ps)$DAI = factor(sample_data(ps)$DAI)

dds = phyloseq_to_deseq2(ps, ~ Rep + SampleType + DAI + SampleType:DAI)

#save variance stabilized counts
vst = varianceStabilizingTransformation(dds, blind=FALSE)


outfile = file.path(work_dir, 'DESeqObject_Exp2CHCS_vsttrans.rds')

saveRDS(vst, file = outfile)

# Checkpoint read vst transformed deseq object
## CHCS

In [ ]:
vstCHCS = readRDS(file.path(work_dir, 'DESeqObject_Exp2CHCS_vsttrans.rds'))

In [ ]:
#get list of only hyphal samples
CHsamples = sample_data(ps) %>% as(.,"data.frame") %>% filter(SampleType == "CH") %>%  .$Sample %>% as.character()

##Cut matrix to hyphal samples and get rownames for most abundant taxa in CH samples
mat = assay(vstCHCS[HASVs,])

topHASVs = mat[order(rowSums(mat),decreasing=T),] %>% rownames() %>%.[1:50]
topHASVs

In [ ]:

# get list to arrange columns of matrix

col.order = sample_data(ps) %>% as(.,"data.frame") %>%  arrange(SampleType,DAI) %>% .$Sample %>% as.character()

# filter to ASVs of interest and arrange according to taxonomy
# this order is not used in final figure

row.order = taxa %>% 
    filter(ASV %in% topHASVs) %>% 
    arrange(Kingdom, Phylum, Class, Order, Family, Genus) %>%
    .$ASV %>% as.character()


# convert variance stabilized counts to matrix
mat = assay(vstCHCS[row.order,col.order])
#mat = mat - rowMeans(mat)
#head(mat)

#Create dataframe for row annotations 
PhylumAnno = rownames(mat) %>% 
    as.data.frame() %>% 
    'colnames<-'('ASV') %>%
    left_join(., taxa, by = "ASV") %>% 
    'rownames<-'(.$ASV) %>% 
    select(Phylum)

# Create dataframe for row labels
LabelAnno = rownames(mat) %>% 
    as.data.frame() %>% 
    'colnames<-'('ASV') %>%
    left_join(., taxa, by = "ASV") %>% 
    'rownames<-'(.$ASV) %>% 
    unite_(col='Label', c("Family","Genus"), sep="_", remove=FALSE) %>%
    select(Label)

#Create dataframe for row annotations based on phylum

#create annotation dataframe
anno = sample_data(ps) %>% as(.,'data.frame') %>% arrange(SampleType, DAI) %>% .[,c("SampleType","DAI")]
#colnames(anno) = 'DAI'
rownames(anno) = colnames(mat)

#anno <- as.data.frame(colData(vst)[, 'DAI'])
ph = pheatmap(mat, cluster_cols = F, cluster_rows = F, annotation_col = anno, annotation_row = PhylumAnno, 
         labels_row = LabelAnno$Label, labels_col = anno$DAI, gaps_col = c(6,12,18, 23, 29, 35, 41, 47, 53), cutree_rows = 4)

pdf(file = file.path(figDir,'HeatMap_Time_CSCH.pdf'), width = 6.89, height = 7)
ph
dev.off()
#pheatmap(mat, annotation_col = anno)


In [ ]:
# get list to arrange columns of matrix

sample.order = sample_data(ps) %>% as(.,"data.frame") %>%  arrange(desc(SampleType),DAI) %>% .$Sample %>% as.character()

# filter to ASVs of interest and arrange according to taxonomy
tax.order = taxa %>% 
    filter(ASV %in% topHASVs) %>% 
    arrange(Kingdom, Phylum, Class, Order, Family, Genus) %>%
    .$ASV %>% as.character()


# convert variance stabilized counts to matrix
mat = t(assay(vstCHBS[tax.order,sample.order]))
#mat = mat - rowMeans(mat)
head(mat)

#Create dataframe for row annotations 
PhylumAnno = colnames(mat) %>%
     as.data.frame() %>% 
     'colnames<-'('ASV') %>%
   left_join(., taxa, by = "ASV") %>% 
    'rownames<-'(.$ASV) %>% 
    select(Phylum)
PhylumAnno

# Create dataframe for taxa labels
LabelAnno = colnames(mat) %>% 
    as.data.frame() %>% 
    'colnames<-'('ASV') %>%
    left_join(., taxa, by = "ASV") %>% 
    'rownames<-'(.$ASV) %>% 
    unite_(col='Label', c("Family","Genus"), sep="_", remove=FALSE) %>%
    select(Label)
LabelAnno

In [ ]:
#create annotation dataframe
anno = sample_data(ps) %>% as(.,'data.frame') %>% arrange(SampleType, DAI) %>% .[,c("SampleType","DAI")]
anno
#colnames(anno) = 'DAI'
#rownames(anno) = colnames(mat)

In [ ]:
ph = pheatmap(mat, cluster_cols = T, cluster_rows = F, labels_col = LabelAnno$Label,
             annotation_col = PhylumAnno, annotation_row = anno, labels_row = anno$DAI, 
              gaps_row = c(6,12,18, 23, 29, 35, 41, 47, 53), cutree_cols = 6) 

In [ ]:

pdf(file = file.path(figDir,'HeatMap_Time_CSCH_horizontal.pdf'), width = 8.086, height = 6)
ph
dev.off()
#pheatmap(mat, annotation_col = anno)

In [ ]:
map.order = colnames(mat[,ph$tree_col[["order"]]])
map.order

In [ ]:
tmp = df.l2fc.Exp3 %>% filter(OTU %in% tax.order & Contrast == 'CH_CS') %>%
        mutate(sig = ifelse(padj < 0.05 & log2FoldChange > 0, 1, 
                           ifelse(padj < 0.05 & log2FoldChange <0, 2, 3))) %>%
        mutate(sig = ifelse(is.na(sig), 3, sig)) %>%
        select(OTU, DAI, sig) %>%
        dcast(DAI~OTU)

rownames(tmp) = tmp$DAI
tmp = tmp %>% select(-DAI)
tmp

In [ ]:
sig.mat = as.matrix(tmp[,map.order])
sig.mat

#pheatmap(sig.mat) 
ph2 = pheatmap(sig.mat, cluster_cols = F, cluster_rows = F, color = c("Red", "Yellow", "White"), breaks = c(0,1,2,3),
             annotation_col = PhylumAnno,
              gaps_col = c(15,26,30,40,45))

In [ ]:
pdf(file = file.path(figDir,'HeatMap_TimeSig_CSCH_horizontal.pdf'), width = 7.75, height = 1.3)
ph2
dev.off()

In [ ]:


pdf(file = file.path(figDir,'HeatMap_Time_CSCH_horizontal.pdf'), width = 8.086, height = 6)
ph
dev.off()
#pheatmap(mat, annotation_col = anno)